In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.naive_bayes import *
from sklearn.dummy import *
from sklearn.ensemble import *
from sklearn.neighbors import *
from sklearn.tree import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.calibration import *
from sklearn.linear_model import *
from sklearn.multiclass import *
from sklearn.svm import *

In [4]:
df = pd.read_csv('data.csv')

df.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [8]:
df.drop_duplicates(inplace = True)
df.shape

(5695, 2)

In [9]:
pd.DataFrame(df.isnull().sum())

,0
text,0
spam,0


In [11]:
df = df.sample(frac=1).reset_index(drop=True)

In [14]:
df

,text,spam
0,Subject: re : credit . com cv ' s my feedback...,0
1,Subject: thank you graham pl . # 207 austin ...,0
2,Subject: re : lng may 19 decision vince - tha...,0
3,Subject: your commissions of $ 5000 per week !...,1
4,Subject: re : new computer jarod : can you h...,0
...,...,...
5690,"Subject: re : next visit to houston george , ...",0
5691,Subject: re : brazil fyi . this is a deal i '...,0
5692,"Subject: localized software , all languages av...",1
5693,Subject: re : maureen raymond - castaneda exte...,0


In [15]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [16]:
lmtzr = WordNetLemmatizer()
with open('english.txt', 'r') as f:
  text = f.readlines()
eng_words_ubuntu = set([lmtzr.lemmatize(x.strip().lower().replace('\'s', '')) for x in text] )
stopwords = set(stopwords.words('english'))
words = eng_words_ubuntu.difference(stopwords)
print(len(words))


56622


In [17]:
tr = re.sub('[^a-zA-Z0-9]+', ' ',df['text'][0])
tr = tr.lower().split()
tr = ' '.join(list(filter(lambda text:text in words,tr)))


In [18]:
tr

'subject credit cv feedback philip excellent cv background university chicago work experience long term capital hopefully learned something somewhat idealistic aim make bid product could really address would handle nevertheless bryan fit role seemed somewhat worked variety specific view mind would like seemed academically oriented might hard communicate work see working see leading group either would good person brainstorm forwarded bryan vince j subject credit cv could talk monday obtain feedback team thanks assistance vince j rosalinda vince j shirley bryan lon subject credit cv rosie making bring houston interview vince rosalinda vince j shirley subject credit cv vince asked forward rosie forwarded rosalinda louise rosalinda subject credit cv rosalinda work melanie doyle london two trying arrange bryan come see vince however finding difficult end would possible forward cv vince assistant sure correct person thanks help need number louise'

In [19]:
df['text'] = df['text'].map(lambda text:re.sub('[^a-zA-Z0-9]+', ' ',text)).apply(lambda x: (x.lower()).split())
df['text']= df['text'].apply(lambda text_list:' '.join(list(filter(lambda text:text in words,text_list))))

In [20]:
df.text[0]

'subject credit cv feedback philip excellent cv background university chicago work experience long term capital hopefully learned something somewhat idealistic aim make bid product could really address would handle nevertheless bryan fit role seemed somewhat worked variety specific view mind would like seemed academically oriented might hard communicate work see working see leading group either would good person brainstorm forwarded bryan vince j subject credit cv could talk monday obtain feedback team thanks assistance vince j rosalinda vince j shirley bryan lon subject credit cv rosie making bring houston interview vince rosalinda vince j shirley subject credit cv vince asked forward rosie forwarded rosalinda louise rosalinda subject credit cv rosalinda work melanie doyle london two trying arrange bryan come see vince however finding difficult end would possible forward cv vince assistant sure correct person thanks help need number louise'

In [12]:
df.to_csv('spam.csv', index=False)

In [21]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(df['text'],df['spam'], test_size = 0.20, shuffle=True)

In [25]:
ytrain

2591    0
1713    0
978     0
1190    0
2987    0
       ..
39      1
2584    1
3524    0
2218    1
70      0
Name: spam, Length: 4556, dtype: int64

In [22]:
def perform(classifiers, vectorizers):
    max_score = 0
    max_name = 0
    for classifier in classifiers:
        for vectorizer in vectorizers:
        
            # train
            vectorize_text = vectorizer.fit_transform(xtrain)
            classifier.fit(vectorize_text, ytrain)

            # score
            vectorize_text = vectorizer.transform(xtest)
            score = classifier.score(vectorize_text, ytest)
            name = classifier.__class__.__name__ + ' with ' + vectorizer.__class__.__name__ 
            print(name, score)
        if score > max_score:
            max_score = score
            max_name = name
    print ('===========================================')
    print ('===========================================')
    print (max_name, max_score)
    print ('===========================================')
    print ('===========================================')

list of various classifiers we are going to use

In [15]:
classifiers = [
        BernoulliNB(),
        MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
        RandomForestClassifier(n_estimators=100, n_jobs=-1),
        AdaBoostClassifier(),
        BaggingClassifier(),
        ExtraTreesClassifier(),
        GradientBoostingClassifier(),
        DecisionTreeClassifier(),
        CalibratedClassifierCV(),
        PassiveAggressiveClassifier(),
        RidgeClassifier(),
        RidgeClassifierCV(),
        SGDClassifier(),
        OneVsRestClassifier(SVC(kernel='linear')),
        OneVsRestClassifier(LogisticRegression()),
        KNeighborsClassifier()
    ]

list of various vectorizers we are going to use

In [16]:
vectorizers = [
        CountVectorizer(),
        TfidfVectorizer(),
        HashingVectorizer()
    ]

perform classification and save results to a new dataframe

In [ ]:
perform(
    classifiers,
    vectorizers
)

BernoulliNB with CountVectorizer 0.9859525899912204
BernoulliNB with TfidfVectorizer 0.9859525899912204
BernoulliNB with HashingVectorizer 0.7576821773485514
RandomForestClassifier with CountVectorizer 0.9762949956101844
RandomForestClassifier with TfidfVectorizer 0.9780509218612818
RandomForestClassifier with HashingVectorizer 0.9508340649692713
AdaBoostClassifier with CountVectorizer 0.9543459174714662
AdaBoostClassifier with TfidfVectorizer 0.9578577699736611
AdaBoostClassifier with HashingVectorizer 0.9578577699736611
BaggingClassifier with CountVectorizer 0.9596136962247586
BaggingClassifier with TfidfVectorizer 0.9596136962247586
BaggingClassifier with HashingVectorizer 0.9569798068481123


d:\desktop\project\venv\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ExtraTreesClassifier with CountVectorizer 0.9657594381035997
ExtraTreesClassifier with TfidfVectorizer 0.9596136962247586
ExtraTreesClassifier with HashingVectorizer 0.9429323968393327
GradientBoostingClassifier with CountVectorizer 0.9683933274802459
GradientBoostingClassifier with TfidfVectorizer 0.9683933274802459
GradientBoostingClassifier with HashingVectorizer 0.9675153643546971
DecisionTreeClassifier with CountVectorizer 0.9473222124670764
DecisionTreeClassifier with TfidfVectorizer 0.9543459174714662
DecisionTreeClassifier with HashingVectorizer 0.9534679543459175


d:\desktop\project\venv\lib\site-packages\sklearn\model_selection\_split.py:2052: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
d:\desktop\project\venv\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


CalibratedClassifierCV with CountVectorizer 0.9780509218612818


...

SGDClassifier with TfidfVectorizer 0.9912203687445127
SGDClassifier with HashingVectorizer 0.9929762949956101
OneVsRestClassifier with CountVectorizer 0.9824407374890255
OneVsRestClassifier with TfidfVectorizer 0.9912203687445127
OneVsRestClassifier with HashingVectorizer 0.9920983318700615


d:\desktop\project\venv\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


OneVsRestClassifier with CountVectorizer 0.9868305531167691


d:\desktop\project\venv\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


OneVsRestClassifier with TfidfVectorizer 0.9754170324846356


In [120]:
# train the classifier with best accuracy
Classifier = OneVsRestClassifier(SVC(kernel='linear', probability=True))
Vectorizer = TfidfVectorizer()
vectorize_text = Vectorizer.fit_transform(train_data.SMS)
Classifier.fit(vectorize_text, train_data.label)

OneVsRestClassifier(estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=True, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
          n_jobs=None)

In [121]:
SMS = ' won a 1 week FREE membership in our $100,000 Prize Jackpot! Txt the word: C'
vectorize_message = Vectorizer.transform([SMS])
predict = Classifier.predict(vectorize_message)[0]

In [122]:
if predict == 0:
    print ('ham')
else:
    print ('spam')

spam
